In [1]:
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision import datasets 
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [2]:
train_path = '/StudentData/hw2_data/train_images/train'
test_path = '/StudentData/hw2_data/test_images/test'

In [3]:
# Hyper Parameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [4]:
# Create Data Loader for Image

class ImageFolderWithName(Dataset):
    def __init__(self, root, transform):
        self.transform = transform
        self.images_paths = glob(root + '/*.jpg')
        self.labels = torch.LongTensor([int(basename(image_path).split('_')[1].split('.')[0]) for image_path in self.images_paths])

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        image_path = self.images_paths[idx]
        image = self.transform(Image.open(image_path))
        label = self.labels[idx]
        return (image, label, basename(image_path))

In [5]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])


transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))  
])    

root_train = '/StudentData/hw2_data/train_images/'
root_test = '/StudentData/hw2_data/test_images/'
image_datasets_train_1 = ImageFolderWithName(root_train, transform=transform_train)
image_datasets_train_2 = ImageFolderWithName(root_train, transform=transform_test)
image_datasets_test = ImageFolderWithName(root_test, transform=transform_test)

train_loader_1 = data.DataLoader(image_datasets_train_1,
                                batch_size=batch_size,
                                shuffle=True) 

train_loader_2 = data.DataLoader(image_datasets_train_2,
                                batch_size=batch_size,
                                shuffle=False) 

test_loader = data.DataLoader(image_datasets_test,
                                batch_size=batch_size,
                                shuffle=False)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(), 
            nn.BatchNorm2d(16)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )
            
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2))
  
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2))
    
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AvgPool2d(2))
    
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2))
        
        self.layer8 = nn.Sequential(
            nn.Conv2d(64, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.AvgPool2d(2))
        
        self.linearLayers = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=144, out_features=2))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = out.view(out.size(0), -1)
        out = self.linearLayers(out)  
        return out

In [7]:
def to_gpu(x):
    return x.cuda() if torch.cuda.is_available() else x

In [8]:
def evaluation_acc(labels, outputs):
    return accuracy_score(labels, outputs)

In [9]:
def evaluation_ROC(labels, outputs):
    return roc_auc_score(labels, outputs)

In [10]:
def evaluation_F1(labels, outputs):
    return f1_score(labels, outputs, average='binary')

In [11]:
cnn = CNN()
cnn = to_gpu(cnn)

criterion = to_gpu(nn.CrossEntropyLoss())
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

print('number of parameters: ', sum(param.numel() for param in cnn.parameters()))
print(f'Num of trainable parameters : {sum(p.numel() for p in cnn.parameters() if p.requires_grad)}')

number of parameters:  698642
Num of trainable parameters : 698642


In [12]:
loss_test_arr = []
loss_train_arr = []
acc_train_arr = []
acc_test_arr = []
ROC_train_arr = []
ROC_test_arr = []
F1_train_arr = []
F1_test_arr = []
names_arr = []
for epoch in range(num_epochs):
    batchs_loss_train = []
    batchs_loss_test = []
    batchs_acc_train = []
    batchs_acc_test = []
    labels_train_arr = []
    labels_test_arr = []
    predicted_train = []
    predicted_test = []
    cnn.train()
    for i, (images, labels, names) in enumerate(train_loader_1):
        images = to_gpu(images)
        labels = to_gpu(labels)
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 200 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(image_datasets_train_1)//batch_size, loss.item()))
    
    cnn.eval()
    for images_test, labels_test, names_test in train_loader_2:
        images_test = to_gpu(images_test)
        labels_test = to_gpu(labels_test)
        outputs_test = cnn(images_test)
        foo, predicted = torch.max(outputs_test.data, 1)
        predicted_train = np.append(predicted_train, predicted.cpu().numpy())
        labels_train_arr = np.append(labels_train_arr, labels_test.cpu().numpy())
        loss_test = criterion(outputs_test, labels_test)
        batchs_loss_train.append((loss.item())*len(images_test))
    
    loss_train_arr.append(sum(batchs_loss_train) / len(image_datasets_train_2))
    acc_train_arr.append(evaluation_acc(labels_train_arr, predicted_train))
    ROC_train_arr.append(evaluation_ROC(labels_train_arr, predicted_train))
    F1_train_arr.append(evaluation_F1(labels_train_arr, predicted_train))
    print("train acc: ", acc_train_arr[-1])
    
    for images_test, labels_test, names_test in test_loader:
        images_test = to_gpu(images_test)
        labels_test = to_gpu(labels_test)
        outputs_test = cnn(images_test)
        foo, predicted = torch.max(outputs_test.data, 1)
        predicted_test = np.append(predicted_test, predicted.cpu().numpy())
        labels_test_arr = np.append(labels_test_arr, labels_test.cpu().numpy())
        names_arr.append(names_test)
        loss_test = criterion(outputs_test, labels_test)
        batchs_loss_test.append((loss_test.item())*len(images_test))
    
    loss_test_arr.append(sum(batchs_loss_test) / len(image_datasets_test))
    acc_test_arr.append(evaluation_acc(labels_test_arr, predicted_test))
    ROC_test_arr.append(evaluation_ROC(labels_test_arr, predicted_test))
    F1_test_arr.append(evaluation_F1(labels_test_arr, predicted_test))
    print("test acc: ", acc_test_arr[-1])

Epoch: [1/10], Step: [200/570], Loss: 0.4870
Epoch: [1/10], Step: [400/570], Loss: 0.2336
train acc:  0.9219015280135824
test acc:  0.9277029247453171
Epoch: [2/10], Step: [200/570], Loss: 0.1767
Epoch: [2/10], Step: [400/570], Loss: 0.1991
train acc:  0.9034996440111726
test acc:  0.9074926059809398
Epoch: [3/10], Step: [200/570], Loss: 0.2604
Epoch: [3/10], Step: [400/570], Loss: 0.2868
train acc:  0.9284736294430144
test acc:  0.9362471245481433
Epoch: [4/10], Step: [200/570], Loss: 0.1799
Epoch: [4/10], Step: [400/570], Loss: 0.2984
train acc:  0.9361958486225971
test acc:  0.9410121590535656
Epoch: [5/10], Step: [200/570], Loss: 0.2225
Epoch: [5/10], Step: [400/570], Loss: 0.1499
train acc:  0.941179692206583
test acc:  0.9477489319750246
Epoch: [6/10], Step: [200/570], Loss: 0.1831
Epoch: [6/10], Step: [400/570], Loss: 0.1325
train acc:  0.9479708636836629
test acc:  0.9465987512323365
Epoch: [7/10], Step: [200/570], Loss: 0.1886
Epoch: [7/10], Step: [400/570], Loss: 0.1717
train

In [13]:
import pickle
with open("loss_test7.txt", "wb") as fp:   #Pickling
    pickle.dump(loss_test_arr, fp)
with open("loss_train7.txt", "wb") as fp:   #Pickling
    pickle.dump(loss_train_arr, fp)
with open("acc_train7.txt", "wb") as fp:     
    pickle.dump(acc_train_arr, fp)
with open("acc_test7.txt", "wb") as fp: 
    pickle.dump(acc_test_arr, fp)
with open("ROC_train7.txt", "wb") as fp: 
    pickle.dump(ROC_train_arr, fp)
with open("ROC_test7.txt", "wb") as fp: 
    pickle.dump(ROC_test_arr, fp)
with open("F1_train7.txt", "wb") as fp: 
    pickle.dump(F1_train_arr, fp)
with open("F1_test7.txt", "wb") as fp: 
    pickle.dump(F1_test_arr, fp)

In [14]:
torch.save(cnn.state_dict(), "cnn_better_model.pt")